In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scvi
import scanpy as sc
import matplotlib
import csv
import os
seed=10
os.chdir(".../Atlas/objects")
sc.logging.print_versions()

Global seed set to 0


-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                 9.2.0
absl                NA
asttokens           NA
attr                22.1.0
backcall            0.2.0
bottleneck          1.3.5
cffi                1.15.1
chex                0.1.5
colorama            0.4.5
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.3
decorator           4.4.2
defusedxml          0.7.1
deprecate           0.3.2
docrep              0.3.2
entrypoints         0.4
etils               0.8.0
executing           1.0.0
flax                0.6.0
fsspec              2022.8.2
google              NA
h5py                3.7.0
igraph              0.9.11
ipykernel           6.15.3
ipython_genutils    0.2.0
ipywidgets          8.0.2
jax                 0.3.17
jaxlib              0.3.15
jedi                0.18.0
joblib              1.2.0
jupyter_server      2.0.4
kiwisolver          1.4.4
leidenalg           0.8.10
llvmlite            0.39.1
louvain          

In [ ]:
#adata 6.2
adata = scvi.data.read_h5ad('Atlas6.2_3000HVG_integrated_V2.h5ad')


In [3]:
condition_list = adata.obs.condition_harmonized.unique().tolist()

In [ ]:
#we check if all conditions / samples equally contribute to clusters or if we have problematic samples 

condition_list = adata.obs.condition_harmonized.unique().tolist()

cell_colors = {
        '1.db/m': "red", '2.db/db': "red", '4b.db/db+AAV+ACEi(2w)': "red", '3b.db/db+AAV+PBS(2w)': "red", '7b.db/db+AAV+SGLT2i(2w)': "red", '5b.db/db+AAV+Rosi(2w)': "red", 'Control': "red", '4hours': "red", '12hours': "red", '2days': "red", '14days': "red", '6weeks': "red", 'G2NA43': "red", 'LYZ2_eYFP': "red", 'FA3': "red", 'WT40': "red", 'CNU': "red", 'GGT_eYFP': "red", 'KD1': "red", 'PDGFR-eYFP': "red", 'APOL1': "red", 'Lean': "red", 'Obese': "red", 'Obese+sGCact': "red", 'Obese+sGCstim': "red", 'DOCA': "red", 'Spironolactone': "red", 'Finerenone': "red", 'Amiloride': "red", 'DKD': "red", 'H-CKD': "red", 'AKI': "red",
    }


for i in range(len(condition_list)):
    
    sc.pl.umap(adata, color=["condition_harmonized"], palette = cell_colors, groups = condition_list[i])


In [4]:
#we further calculate percentages to identify outlier samples

cell_type_counts = adata.obs.annotation_Atlas_level3.value_counts()

cell_type_percentages = cell_type_counts / cell_type_counts.sum() * 100

cell_type_data = cell_type_percentages.to_frame(name='Percentage')

In [6]:
for i in range(len(condition_list)):
    
    adat = adata[adata.obs.condition_harmonized == condition_list[i]]
    
    cell_type_counts = adat.obs.annotation_Atlas_level3.value_counts()
    
    cell_type_percentages = cell_type_counts / cell_type_counts.sum() * 100
    
    cell_type_data[condition_list[i]] = cell_type_percentages

In [8]:
cell_type_data.to_csv(".../Tables/Atlas6.2_QC_afterannot_conditions_PercentCelltype.csv")

In [10]:
ident_list = adata.obs.orig_ident.unique().tolist()

ident_counts = adata.obs.annotation_Atlas_level3.value_counts()

ident_percentage = ident_counts / ident_counts.sum() * 100

ident_data = ident_percentage.to_frame(name='Percentage')

In [ ]:
for i in range(len(ident_list)):
    
    adat = adata[adata.obs.orig_ident == ident_list[i]]
    
    cell_type_counts = adat.obs.annotation_Atlas_level3.value_counts()
    
    cell_type_percentages = cell_type_counts / cell_type_counts.sum() * 100
    
    ident_data[ident_list[i]] = cell_type_percentages

In [13]:
ident_data = ident_data.copy()

In [14]:
ident_data.to_csv(".../Tables/Atlas6.2_QC_afterannot_ident_PercentCelltype.csv")

In [ ]:
#Remove problematic samples 
samples_remove=['KD1', 'WT40', 'APOL1']
adata = adata[~adata.obs['orig_ident'].isin(samples_remove),:]
adata

In [4]:
adata.obs["annotation_Atlas_level0"] = adata.obs["annotation_Atlas_level1"]

Rename_cluster = {
'TAL': "TAL_MD", 'CD_PC': "DCT_CNT_CD", 'DCT_CNT': "DCT_CNT_CD", 'PTS1_PTS2': "PT", 
'CD_Pap': "DCT_CNT_CD", 'ICA': "IC", 'EC': "EC", 'DCT': "DCT_CNT_CD", 'Stromal': "Stromal", 
'injPT': "PT", 'ICB': "IC", 'CNT': "DCT_CNT_CD", 'Immune': "Immune", 'PEC': "PEC", 
'DTL_ATL': "DTL_ATL", 'PTS3': "PT", 'Podo': "Podo", 'PT_prolif': "prolif_Tubule",
}
adata.obs['annotation_Atlas_level0'] = adata.obs['annotation_Atlas_level0'].replace(Rename_cluster).astype('category')



In [5]:
adata.write("Atlas6.3_3000HVG_integrated_V2.h5ad")

In [ ]:
#subclustering: 


#subset clusters of interest 
path_subset = '.../Atlas/Cellnames/Atlas6/Atlas6.3_subset_PT_TAL'
clusters_subset = ["PTS1", "PTS2", "CTAL", "PTS3", "prolif_PT", "injPT", "MTAL", "MD"]
adata_subset = adata[adata.obs['annotation_Atlas_level3'].isin(clusters_subset),:] 
sc.pl.umap(adata_subset, color=['annotation_Atlas_level3'], legend_loc='on data')

cell_names = adata_subset.obs_names
df = pd.DataFrame(cell_names)
df.to_csv(path_subset, columns=[0], header=True)

In [ ]:
#subclustering: 


#subset clusters of interest 
path_subset = '.../Atlas/Cellnames/Atlas6/Atlas6.3_subset_DTL_ATL_injuredPT'
clusters_subset = ["prolif_PT", "injPT", "DTL", "ATL"]
adata_subset = adata[adata.obs['annotation_Atlas_level3'].isin(clusters_subset),:] 
sc.pl.umap(adata_subset, color=['annotation_Atlas_level3'], legend_loc='on data')

cell_names = adata_subset.obs_names
df = pd.DataFrame(cell_names)
df.to_csv(path_subset, columns=[0], header=True)

In [3]:
adata_I = adata[adata.obs.annotation_Atlas_level3 == "Immune"]
adata_S = adata[adata.obs.annotation_Atlas_level3 == "Stromal"]
list_cluster = ["EC", "PT_prolif_unknown_EBF1"]
adata_E = adata[adata.obs.annotation_Atlas_level3.isin(list_cluster)]

In [5]:
sc.pp.neighbors(adata_I, n_pcs=30, use_rep="X_scVI", random_state=seed)
sc.tl.umap(adata_I, min_dist=0.3, random_state=seed)

In [6]:
sc.pp.neighbors(adata_S, n_pcs=30, use_rep="X_scVI", random_state=seed)
sc.tl.umap(adata_S, min_dist=0.3, random_state=seed)

In [7]:
sc.pp.neighbors(adata_E, n_pcs=30, use_rep="X_scVI", random_state=seed)
sc.tl.umap(adata_E, min_dist=0.3, random_state=seed)

In [ ]:
sc.set_figure_params(figsize=(5,5))
sc.pl.umap(adata_I, color=['annotation_Atlas_level3'], legend_loc='on data')
sc.pl.umap(adata_S, color=['annotation_Atlas_level3'], legend_loc='on data')
sc.pl.umap(adata_E, color=['annotation_Atlas_level3'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_I, color=['percent_mt'])
sc.pl.umap(adata_S, color=['percent_mt'])
sc.pl.umap(adata_E, color=['percent_mt'])
sc.pl.umap(adata_I, color=['nCount_RNA'])
sc.pl.umap(adata_S, color=['nCount_RNA'])
sc.pl.umap(adata_E, color=['nCount_RNA'])

In [15]:
marker_gene_list = ["SLC5A2", "SLC5A12", "SLC4A4", "SLC22A6", "SLC13A2", "PCK1", "SLC7A13", "SLC23A1", "ACSS2", "CREB5", "CDH6", "TOP2A", 
"DTL", "FLT1", "PTPRB", "DOCK2", "PRKCB", "CD53", "NPHS1", "NPHS2", "WT1", "ALDH1A2", "FAM189A1", "COL12A1", "LAMA2", "MEIS1",
"EHF", "GRIP1", "PROX1", "BCL6", "ANK2", "EPHA7", "JAG1", "SLC4A11", "AQP2", "SCNN1G", "SCNN1B", "TMEM178A", "ADAMTS16",
"SLC12A1", "TMEM207", "UMOD", "ERBB4", "NOS1", "ENOX1", "LHX1", "TRPM7", "SLC12A3", "TRPM6", "KL", "WNK1", "CNNM2", "CALB1", "SLC8A1",
"IRS1", "SLC4A9", "PDE1C", "SUSD1", "SLC4A1", "SLC26A7", "KIT"]

In [ ]:
sc.pl.umap(adata_I, color=marker_gene_list)

In [ ]:
sc.pl.umap(adata_E, color=marker_gene_list)

In [ ]:
sc.pl.umap(adata_S, color=marker_gene_list)

In [19]:
sc.tl.leiden(adata_I, key_added="leiden_scVI_0_1", resolution=0.1, random_state=seed)
sc.tl.leiden(adata_I, key_added="leiden_scVI_0_3", resolution=0.3, random_state=seed)
sc.tl.leiden(adata_I, key_added="leiden_scVI_0_5", resolution=0.5, random_state=seed)
sc.tl.leiden(adata_I, key_added="leiden_scVI_0_7", resolution=0.7, random_state=seed)
sc.tl.leiden(adata_I, key_added="leiden_scVI_1_0", resolution=1.0, random_state=seed)

In [20]:
sc.tl.leiden(adata_I, key_added="leiden_scVI_1_5", resolution=1.5, random_state=seed)
sc.tl.leiden(adata_I, key_added="leiden_scVI_2_0", resolution=2.0, random_state=seed)

In [21]:
sc.tl.leiden(adata_S, key_added="leiden_scVI_0_1", resolution=0.1, random_state=seed)
sc.tl.leiden(adata_S, key_added="leiden_scVI_0_3", resolution=0.3, random_state=seed)
sc.tl.leiden(adata_S, key_added="leiden_scVI_0_5", resolution=0.5, random_state=seed)
sc.tl.leiden(adata_S, key_added="leiden_scVI_0_7", resolution=0.7, random_state=seed)
sc.tl.leiden(adata_S, key_added="leiden_scVI_1_0", resolution=1.0, random_state=seed)

In [22]:
sc.tl.leiden(adata_S, key_added="leiden_scVI_1_5", resolution=1.5, random_state=seed)
sc.tl.leiden(adata_S, key_added="leiden_scVI_2_0", resolution=2.0, random_state=seed)

In [23]:
sc.tl.leiden(adata_E, key_added="leiden_scVI_0_1", resolution=0.1, random_state=seed)
sc.tl.leiden(adata_E, key_added="leiden_scVI_0_3", resolution=0.3, random_state=seed)
sc.tl.leiden(adata_E, key_added="leiden_scVI_0_5", resolution=0.5, random_state=seed)
sc.tl.leiden(adata_E, key_added="leiden_scVI_0_7", resolution=0.7, random_state=seed)
sc.tl.leiden(adata_E, key_added="leiden_scVI_1_0", resolution=1.0, random_state=seed)

In [24]:
sc.tl.leiden(adata_E, key_added="leiden_scVI_1_5", resolution=1.5, random_state=seed)
sc.tl.leiden(adata_E, key_added="leiden_scVI_2_0", resolution=2.0, random_state=seed)

In [25]:
sc.tl.leiden(adata_E, key_added="leiden_scVI_3_0", resolution=3.0, random_state=seed)

In [ ]:
sc.pl.umap(adata_I, color=['leiden_scVI_0_1'], legend_loc='on data')
sc.pl.umap(adata_I, color=['leiden_scVI_0_3'], legend_loc='on data')
sc.pl.umap(adata_I, color=['leiden_scVI_0_5'], legend_loc='on data')
sc.pl.umap(adata_I, color=['leiden_scVI_0_7'], legend_loc='on data')
sc.pl.umap(adata_I, color=['leiden_scVI_1_0'], legend_loc='on data')
sc.pl.umap(adata_I, color=['leiden_scVI_1_5'], legend_loc='on data')
sc.pl.umap(adata_I, color=['leiden_scVI_2_0'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_S, color=['leiden_scVI_0_1'], legend_loc='on data')
sc.pl.umap(adata_S, color=['leiden_scVI_0_3'], legend_loc='on data')
sc.pl.umap(adata_S, color=['leiden_scVI_0_5'], legend_loc='on data')
sc.pl.umap(adata_S, color=['leiden_scVI_0_7'], legend_loc='on data')
sc.pl.umap(adata_S, color=['leiden_scVI_1_0'], legend_loc='on data')
sc.pl.umap(adata_S, color=['leiden_scVI_1_5'], legend_loc='on data')
sc.pl.umap(adata_S, color=['leiden_scVI_2_0'], legend_loc='on data')

In [ ]:
sc.pl.umap(adata_E, color=['leiden_scVI_0_1'], legend_loc='on data')
sc.pl.umap(adata_E, color=['leiden_scVI_0_3'], legend_loc='on data')
sc.pl.umap(adata_E, color=['leiden_scVI_0_5'], legend_loc='on data')
sc.pl.umap(adata_E, color=['leiden_scVI_0_7'], legend_loc='on data')
sc.pl.umap(adata_E, color=['leiden_scVI_1_0'], legend_loc='on data')
sc.pl.umap(adata_E, color=['leiden_scVI_1_5'], legend_loc='on data')
sc.pl.umap(adata_E, color=['leiden_scVI_2_0'], legend_loc='on data')

In [ ]:
sc.pl.dotplot(
adata_E,
marker_gene_list,
groupby='leiden_scVI_2_0',
dendrogram=True,
color_map="Blues",
swap_axes=False,
standard_scale="var",
)

In [ ]:
sc.pl.dotplot(
adata_E,
marker_gene_list,
groupby='leiden_scVI_2_0',
dendrogram=True,
color_map="Blues",
swap_axes=False,
standard_scale="var",
)

In [ ]:
##remove clusters
clusters_remove=["20", "24", "14", "25", "21"]

adata_E2 = adata_E[~adata_E.obs['leiden_scVI_2_0'].isin(clusters_remove),:]
sc.pl.umap(adata_E2, color="leiden_scVI_2_0")

In [36]:
sc.tl.leiden(adata_E2, key_added="leiden_scVI_0_1", resolution=0.1, random_state=seed)
sc.tl.leiden(adata_E2, key_added="leiden_scVI_0_3", resolution=0.3, random_state=seed)
sc.tl.leiden(adata_E2, key_added="leiden_scVI_0_5", resolution=0.5, random_state=seed)
sc.tl.leiden(adata_E2, key_added="leiden_scVI_0_7", resolution=0.7, random_state=seed)
sc.tl.leiden(adata_E2, key_added="leiden_scVI_1_0", resolution=1.0, random_state=seed)

In [ ]:
sc.pl.umap(adata_E2, color=['leiden_scVI_0_1'], legend_loc='on data')
sc.pl.umap(adata_E2, color=['leiden_scVI_0_3'], legend_loc='on data')
sc.pl.umap(adata_E2, color=['leiden_scVI_0_5'], legend_loc='on data')
sc.pl.umap(adata_E2, color=['leiden_scVI_0_7'], legend_loc='on data')
sc.pl.umap(adata_E2, color=['leiden_scVI_1_0'], legend_loc='on data')

In [ ]:
sc.pl.dotplot(
adata_I,
marker_gene_list,
groupby='leiden_scVI_2_0',
dendrogram=True,
color_map="Blues",
swap_axes=False,
standard_scale="var",
)

In [ ]:
##remove clusters
clusters_remove=['11', "26", "24", "14", "22", "20", "16", "28", "19", "29", "3", "4", "12", "25", "30"]

adata_I2 = adata_I[~adata_I.obs['leiden_scVI_2_0'].isin(clusters_remove),:]
sc.pl.umap(adata_I2, color="leiden_scVI_2_0")

In [ ]:
sc.tl.leiden(adata_I2, key_added="leiden_scVI_0_1", resolution=0.1, random_state=seed)
sc.tl.leiden(adata_I2, key_added="leiden_scVI_0_3", resolution=0.3, random_state=seed)
sc.tl.leiden(adata_I2, key_added="leiden_scVI_0_5", resolution=0.5, random_state=seed)
sc.tl.leiden(adata_I2, key_added="leiden_scVI_0_7", resolution=0.7, random_state=seed)
sc.tl.leiden(adata_I2, key_added="leiden_scVI_1_0", resolution=1.0, random_state=seed)

In [ ]:
sc.pl.umap(adata_I2, color=['leiden_scVI_0_1'], legend_loc='on data')
sc.pl.umap(adata_I2, color=['leiden_scVI_0_3'], legend_loc='on data')
sc.pl.umap(adata_I2, color=['leiden_scVI_0_5'], legend_loc='on data')
sc.pl.umap(adata_I2, color=['leiden_scVI_0_7'], legend_loc='on data')
sc.pl.umap(adata_I2, color=['leiden_scVI_1_0'], legend_loc='on data')

In [ ]:
sc.pl.dotplot(
adata_S,
marker_gene_list,
groupby='leiden_scVI_2_0',
dendrogram=True,
color_map="Blues",
swap_axes=False,
standard_scale="var",
)

In [ ]:
##remove clusters - not removing too many cells here - epitelial/mesench transition!
clusters_remove=["26"]

adata_S2 = adata_S[~adata_S.obs['leiden_scVI_2_0'].isin(clusters_remove),:]
sc.pl.umap(adata_S2, color="leiden_scVI_2_0")

In [ ]:
sc.tl.leiden(adata_S2, key_added="leiden_scVI_0_1", resolution=0.1, random_state=seed)
sc.tl.leiden(adata_S2, key_added="leiden_scVI_0_3", resolution=0.3, random_state=seed)
sc.tl.leiden(adata_S2, key_added="leiden_scVI_0_5", resolution=0.5, random_state=seed)
sc.tl.leiden(adata_S2, key_added="leiden_scVI_0_7", resolution=0.7, random_state=seed)
sc.tl.leiden(adata_S2, key_added="leiden_scVI_1_0", resolution=1.0, random_state=seed)

In [ ]:
sc.pl.umap(adata_S2, color=['leiden_scVI_0_1'], legend_loc='on data')
sc.pl.umap(adata_S2, color=['leiden_scVI_0_3'], legend_loc='on data')
sc.pl.umap(adata_S2, color=['leiden_scVI_0_5'], legend_loc='on data')
sc.pl.umap(adata_S2, color=['leiden_scVI_0_7'], legend_loc='on data')
sc.pl.umap(adata_S2, color=['leiden_scVI_1_0'], legend_loc='on data')

In [46]:
#group and rename

adata_I2.obs['transfer']=adata_I2.obs.leiden_scVI_0_7
Rename_cluster = {
"0": 'Lymph_1',
"1": 'Myeloid_1',
"2": 'Myeloid_2',
"3": 'B',
"4": 'Myeloid_3',
"5": 'Myeloid_4',
"6": 'Lymph_2',
"7": 'Myeloid_5',
"8": 'Myeloid_prolif',
"9": 'Plasma',
"10": 'Myeloid_6',
"11": 'Lymph_prolif',
}
adata_I2.obs['transfer'] = adata_I2.obs['transfer'].replace(Rename_cluster).astype('category')



In [47]:
#group and rename

adata_E2.obs['transfer']=adata_E2.obs.leiden_scVI_0_3
Rename_cluster = {
"0": 'EC1',
"1": 'EC2',
"2": 'EC3',
"3": 'EC4',
"4": 'EC5',
"5": 'EC6',
"6": 'EC7',
}
adata_E2.obs['transfer'] = adata_E2.obs['transfer'].replace(Rename_cluster).astype('category')

In [48]:
#group and rename

adata_S2.obs['transfer']=adata_S2.obs.leiden_scVI_0_7
Rename_cluster = {
"0": 'Str1',
"1": 'Str2',
"2": 'Str3',
"3": 'Str4',
"4": 'Str5',
"5": 'Str6',
"6": 'Str7',
"7": 'Str8',
"8": 'Str9',
"9": 'Str10',
"10": 'Str11',
"11": 'Str12',
"12": 'Str13',
}
adata_S2.obs['transfer'] = adata_S2.obs['transfer'].replace(Rename_cluster).astype('category')

In [ ]:
sc.pl.umap(adata_I2, color="transfer", legend_loc = "on data")
sc.pl.umap(adata_E2, color="transfer", legend_loc = "on data")
sc.pl.umap(adata_S2, color="transfer", legend_loc = "on data")

In [ ]:
adata.obs["annotation_Atlas_level4"] = adata.obs["annotation_Atlas_level3"]
adata

In [97]:
adata.obs["transfer"] = adata_I2.obs.transfer

In [98]:
categs = ['Myeloid_1', 'Myeloid_2', 'B', 'Lymph_1', 
          'Myeloid_5', 'Myeloid_prolif', 'Myeloid_3', 
          'Myeloid_4', 'Myeloid_6', 'Lymph_prolif', 'Lymph_2', 'Plasma']


adata.obs.annotation_Atlas_level4.cat.add_categories(categs, inplace=True)


for i in range(len(categs)):
    # Create a mask indicating which cells have leiden resolution 2 equal to ""
    mask = adata.obs['transfer'] == categs[i]

    # Update the annotation for cells with leiden resolution 2
    adata.obs.loc[mask, 'annotation_Atlas_level4'] = categs[i]

In [100]:
del adata.obs["transfer"]

adata.obs["transfer"] = adata_E2.obs.transfer

categs = ['EC2', 'EC1', 'EC4', 'EC5', 'EC6', 'EC3', 'EC7']


adata.obs.annotation_Atlas_level4.cat.add_categories(categs, inplace=True)


for i in range(len(categs)):
    # Create a mask indicating which cells have leiden resolution 2 equal to ""
    mask = adata.obs['transfer'] == categs[i]

    # Update the annotation for cells with leiden resolution 2
    adata.obs.loc[mask, 'annotation_Atlas_level4'] = categs[i]

In [105]:
del adata.obs["transfer"]

adata.obs["transfer"] = adata_S2.obs.transfer

categs = ['Str8', 'Str1', 'Str5', 'Str9', 'Str3', 'Str6', 'Str2', 'Str7', 'Str10', 'Str4', 'Str11', 'Str13', 'Str12']


adata.obs.annotation_Atlas_level4.cat.add_categories(categs, inplace=True)


for i in range(len(categs)):
    # Create a mask indicating which cells have leiden resolution 2 equal to ""
    mask = adata.obs['transfer'] == categs[i]

    # Update the annotation for cells with leiden resolution 2
    adata.obs.loc[mask, 'annotation_Atlas_level4'] = categs[i]



In [ ]:
##remove clusters (clusters removed in subsets are cells that were removed)
clusters_remove=['Immune', "EC", "Stromal", "PT_prolif_unknown_EBF1"]

adata = adata[~adata.obs['annotation_Atlas_level4'].isin(clusters_remove),:]
adata

In [116]:
#finally save adata again after cleaning 

adata.write('Atlas6.3_3000HVG_integrated_V2.h5ad')